# COMP0005 - GROUP COURSEWORK 2023-24
# Gesture Recognition via Convex Hull 

Use the cell below for all python code needed to realise the **Jarvis march algorithm** (including auxiliary data structures and functions needed by this algorithm - if any). The `jarvismarch()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points (`outputSet`) that lie on the convex hull.

In [76]:
from math import pi, atan2 # for computing polar angle

class GraphUtils():    
    def orientation(a, b, c):
        val = (b[1] - a[1]) * (c[0] - b[0]) - (b[0] - a[0]) * (c[1] - b[1])
        # return 0 if collinear
        if val == 0:
            return 0
        # return 1 if counterclockwise orientation
        elif val > 0:
            return 1
        # return -1 if clockwise orientation
        else:
            return -1
        
    def getMinY(points):
        min = points[0]
        for point in points:
            if point[1] < min[1]:
                min = point
        return min
    
    def dist(a, b):
        xDiffSquare = (a[0] - b[0]) ** 2
        yDiffSquare = (a[1] - b[1]) ** 2
        sumSquare = (xDiffSquare + yDiffSquare) ** 2
        return sumSquare
    
    def angle(a, b):
        angle = atan2(b[1] - a[1], b[0] - a[0]) * (180 / pi)
        if angle < 0:
            angle += 360
        return angle
    
    def isCounterClockWise(self, a, b, c):
        return self.orientation(a,b,c) > 0

    def sort_key(point, P):
        if point == P:
            return float('-inf'), 0
        currAngle = GraphUtils.angle(point, P)
        distance_squared = GraphUtils.dist(point, P)
        return currAngle, distance_squared    

In [106]:
def jarvismarch(inputSet):
    if len(inputSet) <= 3:
        return inputSet

    hull = []
    startPoint = GraphUtils.getMinY(inputSet)
    print(startPoint)
    hull.append(startPoint)
    prev = startPoint
    prevAngle = -1
    while(True):
        minAngle = float('inf')
        maxDist = 0
        next = None
        
        for point in inputSet:
            if  point == prev: 
                continue
            angle = GraphUtils.angle(prev, point)
            dist = GraphUtils.dist(prev, point)
            compareAngles = 0 if angle == minAngle else (-1 if angle < minAngle else 1)
            if compareAngles <= 0 and angle > prevAngle:
                if compareAngles < 0 or dist > maxDist:
                    minAngle = angle
                    maxDist = dist
                    next = point

        if next == startPoint:
            break

        hull.append(next)

        prevAngle = GraphUtils.angle(prev, next)
        prev = next
    
    return hull

points = [(-7,8), (-4,6), (2,6), (6,4), (8,6), (7,-2), (4,-6), (8,-7),(0,0), (3,- 2),(6,-10),(0,6),(-9,-5),(-8,-2),(-8,0),(-10,3),(-2,2),(-10,4)]
# points = [(0, 0), (0, 4), (-4, 0), (5, 0), (0, -6), (1, 0)]
print(jarvismarch(points))


(6, -10)
[(6, -10), (8, -7), (8, 6), (-7, 8), (-10, 4), (-10, 3), (-9, -5)]


Use the cell below for all python code needed to realise the **Graham scan** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `grahamscan()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points that lie on the convex hull (`outputSet`).

In [95]:
import functools 

def turn(a, b, c):
    val = (b[1] - a[1]) * (c[0] - b[0]) - (b[0] - a[0]) * (c[1] - b[1])
    # return 0 if collinear
    if val == 0:
        return 0
    # return 1 if counterclockwise orientation
    elif val > 0:
        return 1
    # return -1 if clockwise orientation
    else:
        return -1

def _keep_left(hull, r):
    while len(hull) > 1 and turn(hull[-2], hull[-1], r) != 1:
            hull.pop()
    return (not len(hull) or hull[-1] != r) and hull.append(r) or hull

def _graham_scan(points):
    """Returns points on convex hull of an array of points in CCW order."""
    points.sort()
    lh = functools.reduce(_keep_left, points, [])
    uh = functools.reduce(_keep_left, reversed(points), [])
    return lh.extend(uh[i] for i in range(1, len(uh) - 1)) or lh

points = [(-7,8), (-4,6), (2,6), (6,4), (8,6), (7,-2), (4,-6), (8,-7),(0,0), (3,- 2),(6,-10),(0,6),(-9,-5),(-8,-2),(-8,0),(-10,3),(-2,2),(-10,4)]
print(_graham_scan(points))

[(-10, 3), (-10, 4), (-7, 8), (8, 6), (8, -7), (6, -10), (-9, -5)]


In [113]:
def grahamscan(inputSet):
    '''
    Returns the list of points that lie on the convex hull (graham scan algorithm)
            Parameters:
                    inputSet (list): a list of 2D points

            Returns:
                    outputSet (list): a list of 2D points
    '''
    if len(inputSet) <= 3:
        return inputSet

    def sort_key(point, P):
        angle = atan2(point[1] - P[1], point[0] - P[0])
        distance_squared = (point[0] - P[0])**2 + (point[1] - P[1])**2
        return (angle, distance_squared)
    
    stack = []
    minYPoint = GraphUtils.getMinY(inputSet)
    inputSet.remove(minYPoint)
    sortedInputSet = sorted(inputSet, key=lambda p: sort_key(p, minYPoint))
    stack.append(minYPoint)
    stack.append(sortedInputSet[0])

    for i in range(1, len(sortedInputSet)):
        next = sortedInputSet[i]
        while len(stack) > 1 and GraphUtils.orientation(stack[-2], stack[-1], next) != -1:
            stack.pop()
        stack.append(next)

    return stack

# points = [(-7,8), (-4,6), (2,6), (6,4), (8,6), (7,-2), (4,-6), (8,-7),(0,0), (3,- 2),(6,-10),(0,6),(-9,-5),(-8,-2),(-8,0),(-10,3),(-2,2),(-10,4)]
points = [(0, 0), (0, 4), (-4, 0), (5, 0), (0, -6), (1, 0)]
# points = [(1,4),(5,6),(2,10),(8,2),(-1,8),(6,3),(7,7),(4,4),(3,9)]

print(grahamscan(points))


[(0, -6), (5, 0), (0, 4), (-4, 0)]


Use the cell below for all python code needed to realise the **Chen's** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `chen()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points that lie on the convex hull (`outputSet`).

In [118]:
def chen(inputSet):    
     
     def tangent(hull, p):
          l = 0 #left most element in the list
          r = len(hull) #right most element in the list
          lPre = GraphUtils.orientation(p, hull[0], hull[-1])
          lSucc = GraphUtils.orientation(p, hull[0], hull[(l + 1) % r])
          while l < r:
               m = l + ((r - l) // 2) #middle element in the list 
               mPre = GraphUtils.orientation(p, hull[m], hull[int((m - 1) % len(hull))])
               mSucc = GraphUtils.orientation(p, hull[m], hull[int((m + 1) % len(hull))])
               mSide = GraphUtils.orientation(p, hull[l], hull[m])
               if mPre != -1 and mSucc != -1:
                    return m
               elif (mSide == 1) and (lSucc == -1 or lPre == lSucc) or (mSide == -1) and (mPre == -1):
                    r = m              
               else:
                    l = m + 1           
                    lPre = -mSucc    
                    lSucc = GraphUtils.orientation(p, hull[l], hull[(l + 1) % len(hull)])
          return l
     
     def minHullPtPair(hulls):
          hull = 0
          pt = 0
          for i in range(len(hulls)):
               j = min(range(len(hulls[i])), key=lambda j: hulls[i][j])
               if hulls[i][j] < hulls[hull][pt]:
                    hull = i
                    pt = j
          return (hull, pt)
     
     def nextHullPtPair(hulls, pair):
          p = hulls[pair[0]][pair[1]]
          next_pair = (pair[0], (pair[1] + 1) % len(hulls[pair[0]]))
          for h in (i for i in range(len(hulls)) if i != pair[0]):
               s = tangent(hulls[h], p)
               q, r = hulls[next_pair[0]][next_pair[1]], hulls[h][s]
               t = turn(p, q, r)
               if t == -1 or (t == 0 and GraphUtils.dist(p, r) > GraphUtils.dist(p, q)):
                    next_pair = (h, s)
          return next_pair

     for m in (1 << (1 << t) for t in range(len(inputSet))):
          hulls = [grahamscan(inputSet[i:i+m]) for i in range(0, len(inputSet), m)]
          hull = [minHullPtPair(hulls)]
          for _ in range(m):
               p = nextHullPtPair(hulls, hull[-1])
               if p == hull[0]:
                    return [hulls[h][i] for h,i in hull]
               hull.append(p)

# points = [(-7,8), (-4,6), (2,6), (6,4), (8,6), (7,-2), (4,-6), (8,-7),(0,0), (3,- 2),(6,-10),(0,6),(-9,-5),(-8,-2),(-8,0),(-10,3),(-2,2),(-10,4)]
points = [(0, 0), (0, 4), (-4, 0), (5, 0), (0, -6), (1, 0)]
# points = [(1,4),(5,6),(2,10),(8,2),(-1,8),(6,3),(7,7),(4,4),(3,9)]


print(chen(points))
         

[(-4, 0), (1, 0), (0, -6)]


Use the cell below to implement the **synthetic data generator** needed by your experimental framework (including any auxiliary data structures and functions you might need - be mindful of code readability and reusability).

In [4]:
import random

class TestDataGenerator():
    """
    A class to represent a synthetic data generator.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    """
        
    #ADD YOUR CODE HERE
    
    def __init__():
        pass


Use the cell below to implement the requested **experimental framework** API.

In [5]:
import timeit
import matplotlib

class ExperimentalFramework():
    """
    A class to represent an experimental framework.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    """
        
    #ADD YOUR CODE HERE
    
    def __init__():
        pass

Use the cell below to illustrate the python code you used to **fully evaluate** the three convex hull algortihms under considerations. The code below should illustrate, for example, how you made used of the **TestDataGenerator** class to generate test data of various size and properties; how you instatiated the **ExperimentalFramework** class to  evaluate each algorithm using such data, collect information about their execution time, plots results, etc. Any results you illustrate in the companion PDF report should have been generated using the code below.

In [6]:
# ADD YOUR TEST CODE HERE 



